In [1]:
import numpy as np
import pandas as pd
import random 

# Simulating evolution 
### (sepcifically) Simulating evolution on a trait that resides on a sex chromosome

## Process:

Classes that we are defining and using:
- fruit fly class, each class object will be an individual:
    - sex
    - individual type: wild/mutated (on hold for now)
    - chromosome1
    - chromosome2
    - expressed_trait
 - action: defines actions that are possible in the environment one of them being sex
     - sex: takes as an input two individuals (male and female) and returns all possible new chromosomal combinations
 - chromosome: contains the chromosomal code
 - fruitfly initializer: intializes the individuals that participate in sex

Chromosome class:

**Entities:**
- chromosomes: type --> panda series
    - 10 characters long for X 
    - 5 characters long for Y
        - an allele will simplistically be represented by a single character on this array, which is simple codon (a simplification of a 3 neuclotide codon) 
        - allele dominance will be an `AND` function
        - We will keep all the other alleleles / codons in the letter (AGCT) form and only the sex linked allele will be given a binary codon - 0/1 
            - this is so that the AND function works 
    - for now we are hardcoding mutation site
    - at the mutation site, a wildtype will have a `0` and a mutant will have codon `1` (to support the AND function we are going to use)

In [2]:
class Chromosome:
    def __init__(self, ctype, is_mutant = False):
        self.ctype = ctype     # this can be X or Y
        self.length = 5 if self.ctype == 'Y' else 10
        self.code = self.initialize()
        self.is_mutant = is_mutant
        self.source = 'wildtype' if is_mutant == False else 'mutant'
        self.mutation_site = 7 if self.ctype == 'X' else None      # setting the mutation site on the chromosome, hardcoding it for now
        self.add_sex_linked_allele()
    
    def initialize(self):
        dna_neuclotides = 'agct'
        data = np.random.randint(0, 4, size=self.length)
        dna_code = [dna_neuclotides[d] for d in data]
        return pd.Series(dna_code)
    
    def add_sex_linked_allele(self):
        if self.ctype == 'Y':
            pass
        else:
            self.code[self.mutation_site] = 0 if self.source == 'wildtype' else 1
        
    
    def print_chromosome(self):
        print(f'Ctype: {self.ctype}')
        print(f'length: {self.length}')
        print(f'source: {self.source}')
        print(f'is_mutant: {self.is_mutant}')
        print(f'\nCode:\n{self.code}')

Fruit fly Class:

Inherits the chromosome class (because chromosomes are required to make an individual)

Attributes:
- sex: user passed and can be `M` or `F`
- is_mutant: 1 or 0 pointing to True or False of if the individual is a mutant type or not
- c1,c2: the two chromosomes that form the two chromatids that make us the 23rd chromosome of the fruit fly individual
    - if the chromosomes are passed in the class definition then those are used to create the indidividual 
    - if no chromosomes are passed then a new individual is randomly generated using the random dna generator in the chromosome class
- expressed_trait_code, expressed_trait: 0--> `white eyes`, and 1--> `red eyes` based on the chromosomes

Funtions in the class:
- initializer class: initializes the class variables 
- initialize chromosomes: is called if the user does not supply a chromosome to craft the individual 
- expressed_trait, trait_decoder: called to check if the individual has white eyes or red eyes 

In [3]:
class Fruit_fly(Chromosome):
    def __init__(self, sex = 'F', is_mutant = False, c1 = None, c2 = None):

        # Case 1: if no chromatids are passed then GENERATE NEW CHROMATIDS to spawn a NEW individual
        if c1 == None and c2 == None:
            self.sex = sex
            self.is_mutant = is_mutant            
            self.c1, self.c2 = self.initialize_chromosomes()
            self.expressed_trait_code = self.express_trait()
        
        # Case 2: If chromatids are passed, then JOIN THE CHROMATIDS to create a new indvidual 
        else:
            self.c1 = c1
            self.c2 = c2
            self.sex = self.sex_checker()
            self.expressed_trait_code = self.express_trait()
            self.is_mutant = bool(self.expressed_trait_code)
            

        self.expressed_trait = self.trait_decoder()

        
    def initialize_chromosomes(self):
        """
        initialize the two chromatids that form an organism 
        using the parent class
        """
        ctid1 = Chromosome('X', is_mutant= self.is_mutant)
        if self.sex == 'F':
            ctid2 = Chromosome('X', is_mutant= self.is_mutant)
        else:
            ctid2 = Chromosome('Y', is_mutant= self.is_mutant)
        return ctid1, ctid2
    
    
    def sex_checker(self):
        if self.c1.ctype == self.c2.ctype:
            return 'F'
        else:
            return 'M'
        
    
    def express_trait(self):
        if self.sex == 'F':
            return self.c1.code[self.c1.mutation_site] & self.c2.code[self.c2.mutation_site]
        
        # because only one of the chromatids will have the mutation
        else:
            try:
                return self.c1.code[self.c1.mutation_site]
            except:
                return self.c2.code[self.c2.mutation_site]
    
    def trait_decoder(self):
        return 'white eyes' if self.expressed_trait_code == 1 else 'red eyes'
    
    def print_properties(self):
        print(f'sex: {self.sex}')
        print(f'is mutant? {self.is_mutant}')
        print(f"expressed_trait: [{self.expressed_trait}]")
        self.c1.print_chromosome()
        self.c2.print_chromosome()

In [4]:
class Sex(Fruit_fly):
    def __init__(self, ind1, ind2):
        self.ind1 = ind1
        self.ind2 = ind2
        
        # if they are of the opposite sex, then let them create new individuals
        if self.gender_checker():
            self.offspring = list(self.sexual_genesis())
            # self.off0, self.off1, self.off2, self.off3 = self.sexual_genesis()
        else:
            print('Sexually Incompatible')
    
    def sexual_genesis(self):
        return Fruit_fly(c1=self.ind1.c1, c2=self.ind2.c1), Fruit_fly(c1=self.ind1.c1, c2=self.ind2.c2), \
        Fruit_fly(c1=self.ind1.c2, c2=self.ind2.c1), Fruit_fly(c1=self.ind1.c2, c2=self.ind2.c2)
        # create 4 new individuals
        
    
    def gender_checker(self):
        # check that genders are opposite 
        if self.ind1.sex == self.ind2.sex:
            return False
        else:
            return True
    
    def logfile(self):
        for (i,off) in enumerate(self.offspring):
            print(f"baby #{i}, sex: {str(off.sex)}, trait: {off.expressed_trait}")
            #print(f"baby #{(i)}")
            #off.print_properties()
    
        # number of offspring
        # capture number of males and females
        # number of mutants vs non-mutants

In [ ]:
# this is where the generations will be created and sex will happen
# we will set up each generation into two buckets Male and Female
# the zeroth generations will have individuals created / spawned 
# the subsequent generations will be born from the sex of the male and female species

class Environment(Sex):
    def __init__(self, num_generations):
        
        # TODO: Add an assert statement here to check the num_generations > 1
        self.num_generations = num_generations
        self.generations = self.create_generation()
        self.sexually_active_females, self.sexually_active_males = self.spawn_env()
        self.offspring_females, self.offspring_males = [],[]
        self.start_simulation()

        
    
    def start_simulation(self):
        
        print('---------------------------------------------------- Starting Simulation ----------------------------------------------------\n\n')
        print('====== starting conditions ======\n')
        #self.print_env()

        while len(self.generations) < self.num_generations:
            self.produce_next_gen()
            #self.print_env()
            print('\n\n**************** end of current generation run ****************')
        
        print('\n---------------------------------------------------- End of Simulation ----------------------------------------------------')
            
    
    
    def create_generation(self):
        return []
    
    def spawn_env(self, mutant_in_mix = True):
        # we create the zeroth or the F0 generation as they have to be made
        # the first set will be females
        females, males = [],[]
        females.append(Fruit_fly(sex='F', is_mutant=False))
        males.append(Fruit_fly(sex='M', is_mutant=mutant_in_mix))
        
        return females, males
        



    # Todo: make changes to the print file to account for the sex segregation you are doing 
    def print_env(self):
        # print(self.generations)
        for gen in self.generations:
            print('\n+++++++++++++ gen +++++++++++++')
            for sexes in gen:
                for individuals in sexes:
                    print((individuals.sex, individuals.expressed_trait_code), end = '\t')

    
    ###### Action functions ######
    
    # produces the next generation by creating offspring individuals from the sexually active adults
    # and completes the generation run by moving offspring to the sexually active list
    def produce_next_gen(self): 
        for f in self.sexually_active_females:
            for m in self.sexually_active_males:
                s = Sex(f,m)
                self.sex_segregator(s.offspring)
        
        self.complete_current_gen()    
    
    # at the complete of a gen run we move the offspring to sexually active
    # we move sexually active to gen 
    # (as they are no longer sexually active, they are replaced by their offspring)
    # we empty the offspring list, ready for the next birthing cycle
    def complete_current_gen(self):
        self.generations.append([self.sexually_active_females, self.sexually_active_males])
        self.sexually_active_females, self.sexually_active_males = self.offspring_females, self.offspring_males
        self.offspring_females, self.offspring_males = [],[]
        
    
    # will file the males and females into separate lists 
    def sex_segregator(self, offspring_list):
        for off in offspring_list:
            if off.sex == 'M':
                self.offspring_males.append(off)
            else:
                self.offspring_females.append(off)
    
    
e = Environment(5)      

---------------------------------------------------- Starting Simulation ----------------------------------------------------


====== starting conditions ======



**************** end of current generation run ****************


**************** end of current generation run ****************


**************** end of current generation run ****************


**************** end of current generation run ****************


In [ ]:
class Logger:
    def __init__(self, generations):
        self.generations = generations
        self.aggregate_metrics_calculator()
    
    def aggregate_metrics_calculator(self):
        for gen in self.generations:
            print(f"F: {len(gen[0])} \t\tM: {len(gen[1])}")
            mutant_count, non_mutants = 0,0

            for female in gen[0]:
                if female.expressed_trait_code == 1:
                    mutant_count+=1
                else:
                    non_mutants+=1
            #print(f"mutants: {mutant_count} non_mutants: {non_mutants}")
            
            #mutant_count, non_mutants = self.count_resetter(mutant_count, non_mutants)
            
            for male in gen[1]:
                if male.expressed_trait_code == 1:
                    mutant_count+=1
                else: 
                    non_mutants+=1
            print(f"mutants: {mutant_count} non_mutants: {non_mutants}")
    
    
    def count_resetter(self, a, b):
        return 0,0
                
l = Logger(e.generations)

In [7]:
print("fem_mutants: {:%}".format(16/128))
print("fem_non_mutants: {:%}".format(112/128))
print("mal_mutants: {:%}".format(32/128))
print("mal_non_mutants: {:%}".format(96/128))

fem_mutants: 12.500000%
fem_non_mutants: 87.500000%
mal_mutants: 25.000000%
mal_non_mutants: 75.000000%


In [10]:
pow(16,2)

256